In [1]:
# import libraries
import pandas as pd
import numpy as np

# get the data from csv file to dataframe
raw_data = pd.read_csv('data/amazon_datafiniti.csv', low_memory=False)

# check column name
raw_data.columns

Index(['id', 'dateAdded', 'dateUpdated', 'name', 'asins', 'brand',
       'categories', 'primaryCategories', 'imageURLs', 'keys', 'manufacturer',
       'manufacturerNumber', 'reviews.date', 'reviews.dateAdded',
       'reviews.dateSeen', 'reviews.doRecommend', 'reviews.id',
       'reviews.numHelpful', 'reviews.rating', 'reviews.sourceURLs',
       'reviews.text', 'reviews.title', 'reviews.username', 'sourceURLs'],
      dtype='object')

In [2]:
raw_data.head()

,id,dateAdded,dateUpdated,name,asins,brand,categories,primaryCategories,imageURLs,keys,...,reviews.dateSeen,reviews.doRecommend,reviews.id,reviews.numHelpful,reviews.rating,reviews.sourceURLs,reviews.text,reviews.title,reviews.username,sourceURLs
0,AVqVGZNvQMlgsOJE6eUY,2017-03-03T16:56:05Z,2018-10-25T16:36:31Z,"Amazon Kindle E-Reader 6"" Wifi (8th Generation...",B00ZV9PXP2,Amazon,"Computers,Electronics Features,Tablets,Electro...",Electronics,https://pisces.bbystatic.com/image2/BestBuy_US...,allnewkindleereaderblack6glarefreetouchscreend...,...,"2018-05-27T00:00:00Z,2017-09-18T00:00:00Z,2017...",False,NaN,0,3,http://reviews.bestbuy.com/3545/5442403/review...,I thought it would be as big as small paper bu...,Too small,llyyue,https://www.newegg.com/Product/Product.aspx%25...
1,AVqVGZNvQMlgsOJE6eUY,2017-03-03T16:56:05Z,2018-10-25T16:36:31Z,"Amazon Kindle E-Reader 6"" Wifi (8th Generation...",B00ZV9PXP2,Amazon,"Computers,Electronics Features,Tablets,Electro...",Electronics,https://pisces.bbystatic.com/image2/BestBuy_US...,allnewkindleereaderblack6glarefreetouchscreend...,...,"2018-05-27T00:00:00Z,2017-07-07T00:00:00Z,2017...",True,NaN,0,5,http://reviews.bestbuy.com/3545/5442403/review...,This kindle is light and easy to use especiall...,Great light reader. Easy to use at the beach,Charmi,https://www.newegg.com/Product/Product.aspx%25...
2,AVqVGZNvQMlgsOJE6eUY,2017-03-03T16:56:05Z,2018-10-25T16:36:31Z,"Amazon Kindle E-Reader 6"" Wifi (8th Generation...",B00ZV9PXP2,Amazon,"Computers,Electronics Features,Tablets,Electro...",Electronics,https://pisces.bbystatic.com/image2/BestBuy_US...,allnewkindleereaderblack6glarefreetouchscreend...,...,2018-05-27T00:00:00Z,True,NaN,0,4,https://reviews.bestbuy.com/3545/5442403/revie...,Didnt know how much i'd use a kindle so went f...,Great for the price,johnnyjojojo,https://www.newegg.com/Product/Product.aspx%25...
3,AVqVGZNvQMlgsOJE6eUY,2017-03-03T16:56:05Z,2018-10-25T16:36:31Z,"Amazon Kindle E-Reader 6"" Wifi (8th Generation...",B00ZV9PXP2,Amazon,"Computers,Electronics Features,Tablets,Electro...",Electronics,https://pisces.bbystatic.com/image2/BestBuy_US...,allnewkindleereaderblack6glarefreetouchscreend...,...,2018-10-09T00:00:00Z,True,177283626.0,3,5,https://redsky.target.com/groot-domain-api/v1/...,I am 100 happy with my purchase. I caught it o...,A Great Buy,Kdperry,https://www.newegg.com/Product/Product.aspx%25...
4,AVqVGZNvQMlgsOJE6eUY,2017-03-03T16:56:05Z,2018-10-25T16:36:31Z,"Amazon Kindle E-Reader 6"" Wifi (8th Generation...",B00ZV9PXP2,Amazon,"Computers,Electronics Features,Tablets,Electro...",Electronics,https://pisces.bbystatic.com/image2/BestBuy_US...,allnewkindleereaderblack6glarefreetouchscreend...,...,2018-05-27T00:00:00Z,True,NaN,0,5,https://reviews.bestbuy.com/3545/5442403/revie...,Solid entry level Kindle. Great for kids. Gift...,Solid entry-level Kindle. Great for kids,Johnnyblack,https://www.newegg.com/Product/Product.aspx%25...


In [3]:
raw_data.shape

(5000, 24)

In [5]:
raw_data["is_duplicate"]= raw_data.duplicated()
raw_data.duplicated().sum()

95

In [6]:
#drop duplicate data
raw_data.drop_duplicates(keep="last", inplace=True)
raw_data.shape

(4905, 25)

In [7]:
# check unique username and goods name
user_count = raw_data['reviews.username'].unique().shape[0]
item_count = raw_data['name'].unique().shape[0]

print("user count: " + str(user_count) + ", item count: " + str(item_count))

user count: 3815, item count: 23


In [8]:
# indexing item
itemname = []
item_id = []
start_id = 1

for x in raw_data['name']:
    if itemname:
        for y in itemname:
            if x not in itemname :
                itemname.append(x)
                item_id.append(start_id)
                start_id = start_id + 1
            else:
                break
    else:
        itemname.append(x)
        item_id.append(start_id)
        start_id = start_id + 1

item_table = {
    'itemId':item_id,
    'itemname':itemname
}
item_df= pd.DataFrame(item_table)
# check the item_df
# item_df.head(50)
item_df.dtypes

itemId       int64
itemname    object
dtype: object

In [9]:
raw_data = pd.merge(raw_data, item_df, how='left',
                     left_on='name', right_on='itemname')

In [10]:
data_test = raw_data[
    ['itemId', 
     'reviews.username',
     'reviews.rating', 
     'reviews.text']
]

In [11]:
data_test.head()
data_test.shape

(4905, 4)

In [12]:
# making dataset for testing 250 rows
data_test_filter = pd.DataFrame
data_test_filter = data_test.groupby('itemId').head(12)
data_test_filter.shape

(268, 4)

In [13]:
# to be 250 rows, delete (268-250=18 rows)
count = 1
drop_list = []
for x in data_test_filter.index:
    count +=1
    if count%11==0:
        drop_list.append(count)
len(drop_list)

24

In [14]:
# just need 18, delete the last 4 elements in list
del drop_list[-6:]
len(drop_list)

18

In [15]:
data_test_filter.drop(data_test_filter.index[drop_list],
                      inplace=True)

/home/dea/Documents/projects/RecSys/recenv/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [16]:
data_test_filter.shape

(250, 4)

In [17]:
# indexing username
username = []
user_id = []
start_id = 1

for x in data_test_filter['reviews.username']:
    if username:
        for y in username:
            if x not in username :
                username.append(x)
                user_id.append(start_id)
                start_id = start_id + 1
            else:
                break
    else:
        username.append(x)
        user_id.append(start_id)
        start_id = start_id + 1

user_table = {
    'userId':user_id,
    'username':username
}
user_df = pd.DataFrame(user_table)
# check the user_df
# user_df

In [18]:
# merge user_df and data_test_filter
data_test_filter = pd.merge(
    data_test_filter, user_df, how='left',
    left_on='reviews.username', right_on='username')

In [19]:
data_test_filter.head()

,itemId,reviews.username,reviews.rating,reviews.text,userId,username
0,1,llyyue,3,I thought it would be as big as small paper bu...,1,llyyue
1,1,Charmi,5,This kindle is light and easy to use especiall...,2,Charmi
2,1,johnnyjojojo,4,Didnt know how much i'd use a kindle so went f...,3,johnnyjojojo
3,1,Kdperry,5,I am 100 happy with my purchase. I caught it o...,4,Kdperry
4,1,Johnnyblack,5,Solid entry level Kindle. Great for kids. Gift...,5,Johnnyblack


In [20]:
rating_convert = []
for x in data_test_filter['reviews.rating']:
    if x == 5:
        v = 3
    elif x == 4:
        v = 2.5
    elif x == 3:
        v = 2
    elif x == 2:
        v = 1.5
    else:
        v = 1
    rating_convert.append(v)
data_test_filter['reviews.rating']=rating_convert    

In [21]:
data_test_filter.drop(
    ["reviews.username",
     "username"
    ], 
    axis=1, inplace=True)

In [22]:
data_test_filter.head()

,itemId,reviews.rating,reviews.text,userId
0,1,2.0,I thought it would be as big as small paper bu...,1
1,1,3.0,This kindle is light and easy to use especiall...,2
2,1,2.5,Didnt know how much i'd use a kindle so went f...,3
3,1,3.0,I am 100 happy with my purchase. I caught it o...,4
4,1,3.0,Solid entry level Kindle. Great for kids. Gift...,5


In [23]:
data_test_filter = data_test_filter[
    ['itemId',
     'userId',
     'reviews.rating', 
     'reviews.text']]

In [24]:
data_test_filter.rename(
    columns={
        'reviews.rating': 'rating', 
        'reviews.text': 'comment'}, 
    inplace=True)

In [25]:
data_test_filter.head()

,itemId,userId,rating,comment
0,1,1,2.0,I thought it would be as big as small paper bu...
1,1,2,3.0,This kindle is light and easy to use especiall...
2,1,3,2.5,Didnt know how much i'd use a kindle so went f...
3,1,4,3.0,I am 100 happy with my purchase. I caught it o...
4,1,5,3.0,Solid entry level Kindle. Great for kids. Gift...


In [26]:
data_test_filter.to_csv("data/data_prepared_before_sentiment.csv")

In [58]:
# Start the recommendation 

def favoriteItem(activeUser, N):
    topItem=pd.DataFrame.sort_values(
                data[data.userId==activeUser], ['rating'], ascending=[0])[:N]
    return list(topItem.itemName)

print (favoriteItem(17,3))#3 fav item of user 17


['Amazon Kindle E-Reader 6" Wifi (8th Generation, 2016)', 'Amazon Echo Show Alexa-enabled Bluetooth Speaker with 7" Screen', 'Amazon Echo Show Alexa-enabled Bluetooth Speaker with 7" Screen']
